In [1]:
import pandas as pd
import numpy as np

In [2]:
samples_2017 = pd.read_csv('clue/repurposing_samples_20170327.txt',usecols=['broad_id','pert_iname','InChIKey'],delimiter='\t', comment='!', encoding='iso-8859-1')
drugs_2017 = pd.read_csv('clue/repurposing_drugs_20170327.txt', usecols=['pert_iname','moa','target'], delimiter='\t', comment='!', encoding='iso-8859-1')
samples_2018 = pd.read_csv('clue/repurposing_samples_20180907.txt', usecols=['broad_id','pert_iname','InChIKey'], delimiter='\t', comment='!', encoding='iso-8859-1')
drugs_2018 = pd.read_csv('clue/repurposing_drugs_20180907.txt', usecols=['pert_iname','moa','target'], delimiter='\t', comment='!', encoding='iso-8859-1')
samples_2020 = pd.read_csv('clue/repurposing_samples_20200324.txt', usecols=['broad_id','pert_iname','InChIKey'], delimiter='\t', comment='!', encoding='iso-8859-1')
drugs_2020 = pd.read_csv('clue/repurposing_drugs_20200324.txt', usecols=['pert_iname','moa','target'], delimiter='\t', comment='!', encoding='iso-8859-1')

In [3]:
def target_annotation(samples, drugs):
    samples = samples.merge(drugs, on='pert_iname', how='left')
    return samples

def id_cleanup(df, year):
    df.broad_id = df.broad_id.apply(lambda x: str(x)[:13])
    df.InChIKey = df.InChIKey.apply(lambda x: str(x)[:14])
    df = df.drop_duplicates(['InChIKey','pert_iname','broad_id']).reset_index(drop=True)
    df = df.rename(columns={'pert_iname':'pert_iname_'+year,
                            'broad_id':'broad_id_'+year,
                            'InChIKey':'InChIKey14',
                            'moa':'moa_'+year,
                            'target':'target_'+year})
    return df

def group_by_InChIKey14(df, year):
    df = df.fillna('')
    df = df.groupby('InChIKey14').agg({'broad_id_'+year : lambda x: '|'.join(np.unique(x)),
                                       'pert_iname_'+year: lambda x: '|'.join(np.unique(x)),
                                       'moa_'+year : lambda x: merge_target(list(x)),
                                       'target_'+year: lambda x: merge_target(list(x))}).reset_index()
    return df

def merge_target(target):
    out_target = '|'.join(np.unique(('|'.join(target)).split('|')))
    return out_target

In [4]:
samples_2017 = target_annotation(samples_2017, drugs_2017)
samples_2018 = target_annotation(samples_2018, drugs_2018)
samples_2020 = target_annotation(samples_2020, drugs_2020)

samples_2017.head()

,broad_id,pert_iname,InChIKey,moa,target
0,BRD-A37752546-001-01-9,(1E)-1-(2-hydroxy-5-methylphenyl)-1-dodecanone...,NFONIVRMILHYLH-UHFFFAOYSA-N,NaN,NaN
1,BRD-K89787693-001-01-1,"[sar9,met(o2)11]-substance-p",OUPXSLGGCPUZJJ-SARDKLJWSA-N,NaN,NaN
2,BRD-K88956297-003-01-9,"1-((Z)-3-Chloroallyl)-1,3,5,7-tetraazaadamanta...",LDLCEGCJYSDJLX-UPHRSURJSA-N,NaN,NaN
3,BRD-A86415025-050-01-0,"1-(1,2-Diphenylethyl)piperidine-(+/-)",JQWJJJYHVHNXJH-UHFFFAOYSA-N,NaN,NaN
4,BRD-A95802703-001-01-0,1-(2-chloro-5-methylphenoxy)-3-(isopropylamino...,NJEIOWSBPCZKTL-UHFFFAOYSA-N,NaN,NaN


In [5]:
samples_2017 = id_cleanup(samples_2017, "2017")
samples_2018 = id_cleanup(samples_2018, "2018")
samples_2020 = id_cleanup(samples_2020, "2020")

samples_2017.head()

,broad_id_2017,pert_iname_2017,InChIKey14,moa_2017,target_2017
0,BRD-A37752546,(1E)-1-(2-hydroxy-5-methylphenyl)-1-dodecanone...,NFONIVRMILHYLH,NaN,NaN
1,BRD-K89787693,"[sar9,met(o2)11]-substance-p",OUPXSLGGCPUZJJ,NaN,NaN
2,BRD-K88956297,"1-((Z)-3-Chloroallyl)-1,3,5,7-tetraazaadamanta...",LDLCEGCJYSDJLX,NaN,NaN
3,BRD-A86415025,"1-(1,2-Diphenylethyl)piperidine-(+/-)",JQWJJJYHVHNXJH,NaN,NaN
4,BRD-A95802703,1-(2-chloro-5-methylphenoxy)-3-(isopropylamino...,NJEIOWSBPCZKTL,NaN,NaN


In [6]:
samples_2017 = group_by_InChIKey14(samples_2017, "2017")
samples_2018 = group_by_InChIKey14(samples_2018, "2018")
samples_2020 = group_by_InChIKey14(samples_2020, "2020")

samples_2017.head()

,InChIKey14,broad_id_2017,pert_iname_2017,moa_2017,target_2017
0,AAALVYBICLMAMA,BRD-K13087974,CGP-52411,EGFR inhibitor,EGFR
1,AAAQFGUYHFJNHI,BRD-K08109215,I-BET-762,bromodomain inhibitor,BRD2|BRD3|BRD4
2,AADCDMQTJNYOSS,BRD-K50417881,eticlopride,dopamine receptor antagonist,DRD2|DRD3|DRD4
3,AAFJXZWCNVJTMK,BRD-K68502831,dianhydrogalactitol,DNA alkylating agent,
4,AAGFPTSOPGCENQ,BRD-K10999968|BRD-K66845263,sophocarpine,,


In [7]:
merged_df = samples_2017.merge(samples_2018, on='InChIKey14', how='outer')
merged_df = merged_df.merge(samples_2020, on='InChIKey14', how='outer')
merged_df= merged_df.replace(to_replace='', value=np.nan)

merged_df.head()

,InChIKey14,broad_id_2017,pert_iname_2017,moa_2017,target_2017,broad_id_2018,pert_iname_2018,moa_2018,target_2018,broad_id_2020,pert_iname_2020,moa_2020,target_2020
0,AAALVYBICLMAMA,BRD-K13087974,CGP-52411,EGFR inhibitor,EGFR,BRD-K13087974,CGP-52411,EGFR inhibitor,EGFR,BRD-K13087974,CGP-52411,EGFR inhibitor,EGFR
1,AAAQFGUYHFJNHI,BRD-K08109215,I-BET-762,bromodomain inhibitor,BRD2|BRD3|BRD4,BRD-K08109215,I-BET-762,bromodomain inhibitor,BRD2|BRD3|BRD4,BRD-K08109215,I-BET-762,bromodomain inhibitor,BRD2|BRD3|BRD4
2,AADCDMQTJNYOSS,BRD-K50417881,eticlopride,dopamine receptor antagonist,DRD2|DRD3|DRD4,BRD-K50417881,eticlopride,dopamine receptor antagonist,DRD2|DRD3|DRD4,BRD-K50417881,eticlopride,dopamine receptor antagonist,DRD2|DRD3|DRD4
3,AAFJXZWCNVJTMK,BRD-K68502831,dianhydrogalactitol,DNA alkylating agent,NaN,BRD-K68502831,dianhydrogalactitol,DNA alkylating agent,NaN,BRD-K68502831,dianhydrogalactitol,DNA alkylating agent,NaN
4,AAGFPTSOPGCENQ,BRD-K10999968|BRD-K66845263,sophocarpine,NaN,NaN,BRD-K66845263,sophocarpine,NaN,NaN,BRD-K66845263,sophocarpine,NaN,NaN


In [8]:
merged_df.to_csv('clue/broad_id_map.csv', index=False)
